In [121]:
# Set matplotlib for plotting in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

from fooof import FOOOF
from fooof import FOOOFGroup
import numpy as np

# Import some utilities for synthesizing some test data
from fooof.synth import gen_group_power_spectra, param_sampler, gen_power_spectrum

In [116]:
#fAlpha = FOOOF()
#fThetaBetea = FOOOF()

#Generate Power Spectra
#freqAlpha, powerAlpha = gen_power_spectrum ([0,50],[0,1], [10,1,1], nlv=.05)
#freqThetaBeta, powerThetaBeta = gen_power_spectrum([0,50],[0,1],[4, 1, 1, 25, 1, 1],nlv=.05)

#fAlpha.add_data(freqAlpha,powerAlpha)
#fThetaBetea.add_data(freqThetaBeta,powerThetaBeta)

#Plot power spectra
#fAlpha.plot()
#fThetaBetea.plot()


In [117]:
#fAlpha.fit()
#fThetaBetea.fit()

In [118]:
#fAlpha.plot()
#fThetaBetea.plot()

In [57]:
#tPower = fThetaBetea.peak_params_[0][1]
#bPower = fThetaBetea.peak_params_[1][1]

In [119]:
#print("Theta Power: " + str(tPower))
#print("Beta Power: " + str(bPower))


In [120]:
#print("Theta/Beta Ratio: " +str(tPower/bPower))

## Streamline simulations

In [60]:
#gen_group_power_spectra?

In [122]:
#Creates a single adhd simulated spectrum
# Alpha peaks on lower park of band to potentially bleed over to theta band

def gen_adhd_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 10)) #slow Alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

In [123]:
#Creatsa single control simulated spectrum (Theta, slow peak alpha, beta)
def gen_control_sample():
    sample = []
    
    sample.append(np.random.uniform(4, 7)) #Theta freq
    sample.append(np.random.uniform(.35,.75)) #Theta Amp
    sample.append(np.random.uniform(.25,1.05))#Theta SD
    
    sample.append(np.random.uniform(8, 12)) #slow alpha freq
    sample.append(np.random.uniform(.25,.55)) #Alpha Amp
    sample.append(np.random.uniform(.25,1.05))#Alpha SD
    
    sample.append(np.random.uniform(18, 25)) #Beta freq
    sample.append(np.random.uniform(.4,.75)) #Beta Amp
    sample.append(np.random.uniform(.25,1.05))#Beta SD
    
    
    return sample

In [126]:
#Gen group of ADHD PSDs (slow alpha peak freq) + control PSD
bg = param_sampler([[0,2],[0,1]])

#100 trails for each adhd and control sim
adhdFreq, adhdPower, syn_params_adhd = gen_group_power_spectra(100, [1,50], bg,gen_adhd_sample(), nlvs=np.random.uniform(.005,.02)) 
contFreq, contPower, syn_params_control = gen_group_power_spectra(100, [1,50], bg, gen_control_sample(), nlvs=np.random.uniform(.005,.02))

adhd = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)
control = FOOOFGroup(peak_width_limits=[1,8], min_peak_amplitude=0.05, max_n_peaks=3)

In [127]:
adhd.fit(adhdFreq,adhdPower)
control.fit(contFreq,contPower)

In [128]:
#Gather peak info for both PSDs
adhd_peaks = adhd.get_all_data('peak_params')
control_peaks = control.get_all_data('peak_params')

## Calculate theta/beta ratios

In [129]:
# Adhd (Slow alpha prek)
adhdRatioList = []
for i in range(len(adhd_peaks)-2):
    tbRatioList.append(adhd_peaks[i][1]/adhd_peaks[i+2][1])
    i+=2

In [130]:
#Sanity Check
tbRatioList[0]

0.9816036665807988

In [131]:
# control
controlRatioList = []
for i in range(len(control_peaks)-2):
    controlRatioList.append(control_peaks[i][1]/control_peaks[i+2][1])
    i+=2


In [132]:
#Sanity Check
controlRatioList[0]

1.507328816603785

In [133]:
#Difference in mean ratios of adhd sim and control sim
abs(np.mean(tbRatioList)-np.mean(controlRatioList))

0.011564416086740525

I plan to:
- fiddle with the values in the gen_*_sample functions in this note book
- get a better idea of how noise and randomness affect the difference in the ratios 
